In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
1,Covid: Biden says ‘Neanderthal thinking’ behin...,https://www.bbc.com/news/world-us-canada-56275103,"Thu, 04 Mar 2021 00:49:28 GMT",2021-03-04 00:49:28+00:00
2,ClackCo March 3 COVID-19 update: 3 things to k...,https://www.clackamas.us/news/2021-03-03/clack...,"Thu, 04 Mar 2021 00:16:45 GMT",2021-03-04 00:16:45+00:00
0,An Uncertain Public — Encouraging Acceptance o...,https://www.nejm.org/doi/full/10.1056/NEJMp210...,"Wed, 03 Mar 2021 22:01:03 GMT",2021-03-03 22:01:03+00:00
3,Vaccination plus Decarceration — Stopping Covi...,https://www.nejm.org/doi/full/10.1056/NEJMp210...,"Wed, 03 Mar 2021 22:01:03 GMT",2021-03-03 22:01:03+00:00
7,Governor Cuomo Announces More Than 120K Doses ...,https://www.governor.ny.gov/news/governor-cuom...,"Wed, 03 Mar 2021 20:46:13 GMT",2021-03-03 20:46:13+00:00
4,New Guidance On COVID-19 Testing And Vaccines ...,https://www.healthaffairs.org/do/10.1377/hblog...,"Wed, 03 Mar 2021 20:29:05 GMT",2021-03-03 20:29:05+00:00
5,MSU retains COVID-19 safety protocols through ...,https://www.msstate.edu/newsroom/article/2021/...,"Wed, 03 Mar 2021 20:14:46 GMT",2021-03-03 20:14:46+00:00
9,CDC recommends veterinarians be prioritized fo...,https://www.avma.org/javma-news/2021-03-15/cdc...,"Wed, 03 Mar 2021 15:49:57 GMT",2021-03-03 15:49:57+00:00
6,COVID-19 vaccine FAQs University of Dayton - ...,https://udayton.edu/studev/health_wellness/hea...,"Wed, 03 Mar 2021 05:11:46 GMT",2021-03-03 05:11:46+00:00
8,FDA.gov Coronavirus (COVID-19) Update: March 2...,https://www.fda.gov/news-events/press-announce...,"Tue, 02 Mar 2021 20:45:00 GMT",2021-03-02 20:45:00+00:00
